In [34]:
import numpy as np

def complex_givens_rotation_matrix(m, i, j, a, b, tol=1e-12):
    """Construct a complex Givens rotation matrix G that zeroes out b using a."""
    G = np.eye(m, dtype=np.complex128)
    r = np.hypot(np.abs(a), np.abs(b))
    
    if np.abs(b) < tol:
        return G, 1.0, 0.0  # No rotation needed

    # Normalize (a, b) and include phase of 'a'
    c = a / r
    s = -b / r

    # Fill in the rotation elements
    G[i, i] = np.conj(c)
    G[j, j] = c
    G[i, j] = np.conj(s)
    G[j, i] = s
    return G, c, s

def star_qr_complex_givens(U, central_row=1, tol=1e-12):
    """QR decomposition of a complex unitary using complex Givens in star topology."""
    m, n = U.shape
    V = U.copy()
    rotations = []
    list_givens = []
    for col in range(n):
        for i in range(m - 1, -1, -1):
            if i == central_row:
                continue
            a = V[central_row, col]
            b = V[i, col]
            if np.abs(b) < tol:
                continue

            G, c, s = complex_givens_rotation_matrix(m, central_row, i, a, b, tol)
            list_givens.append(G)
            V = G @ V
            rotations.append((central_row, i, c, s, col))  # rotation parameters

    R = V
    return R, rotations, list_givens


In [98]:
import numpy as np
from typing import Tuple, List

def _givens(cs: float, sn: float, m: int, p: int, q: int) -> np.ndarray:
    """
    Build an m×m Givens rotation that mixes rows p and q with
    cosine ``cs`` and sine ``sn`` (real arithmetic).
    """
    G = np.eye(m, dtype=float)
    G[[p, p, q, q], [p, q, p, q]] = cs, sn, -sn, cs
    return G


def _givens_params(a: float, b: float) -> Tuple[float, float]:
    """
    Return (c, s) such that [[c, s], [-s, c]] @ [a, b]^T = [r, 0]^T.
    """
    if b == 0.0:
        return 1.0, 0.0
    r = np.hypot(a, b)
    return a / r, b / r


def star_qr(A: np.ndarray, central: int = 0, tol: float = 1e-12
            ) -> Tuple[np.ndarray, np.ndarray, List[Tuple[int, int, float, float]]]:
    """
    QR factorisation using a star-topology rotation graph.

    Parameters
    ----------
    A : ndarray (m×n)
        Real matrix to factor.
    central : int, optional
        Index of the row that plays the role of the hub (default 0).
    tol : float, optional
        Threshold below which a value is considered zero.

    Returns
    -------
    Q : ndarray (m×m), orthogonal
    R : ndarray (m×n), upper-triangular
    log : list[(p, q, c, s)]
        Sequence of rotations actually applied (row-pair and (c,s)).
    """
    A = np.array(A, dtype=float, copy=True)
    m, n = A.shape
    # Bring the chosen central row to index 0 (permute rows)
    P = np.eye(m)
    P[[0, central]] = P[[central, 0]]
    A = P @ A

    Q = np.eye(m)
    log: List[Tuple[int, int, float, float]] = []

    for col in range(min(m, n)):
        # 1. Eliminate everything below the central row in column 'col'
        for r in range(m - 1, 0, -1):
            if abs(A[r, col]) < tol:
                continue
            c, s = _givens_params(A[0, col], A[r, col])
            G = _givens(c, s, m, 0, r)
            A = G @ A
            Q = G @ Q
            log.append((0, r, c, s))

        # 2. Swap row 0 with row 'col' so we can zero the next column
        if col + 1 < m:
            A[[0, col + 1]] = A[[col + 1, 0]]
            Q[[0, col + 1]] = Q[[col + 1, 0]]
            log.append(("swap", 0, col + 1))

    # Undo the initial permutation so outputs align with original row order
    Q = P.T @ Q.T         # Q was built as product of G, so transpose it
    R = A                 # upper-triangular already
    Q = Q.T               # make Q orthogonal (so that Q @ R = original A)

    return Q, R, log


A = np.real(np.array([[0.354-0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,
   0.354+0.j,  0.354+0.j,  0.354+0.j],
 [0.354-0.j, -0.354+0.j,  0.354+0.j, -0.354-0.j,  0.354+0.j,
  -0.354-0.j,  0.354+0.j, -0.354-0.j],
 [0.354-0.j,  0.354-0.j, -0.354-0.j, -0.354+0.j,  0.354+0.j,
   0.354+0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j,  0.354+0.j,
  -0.354-0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j,  0.354-0.j,  0.354-0.j,  0.354-0.j, -0.354-0.j,
  -0.354-0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j, -0.354+0.j,
   0.354+0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j, 0.354-0.j, -0.354+0.j, -0.354+0.j, -0.354+0.j,
  -0.354+0.j,  0.354+0.j, 0.354+0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j,
   0.354+0.j,  0.354+0.j, -0.354+0.j]],  dtype=complex))
Q, R, log = star_qr(A, central=7)   # make row 2 the hub
print(np.allclose(Q @ R, A))        # → True
print(np.linalg.norm(Q.T @ Q - np.eye(8)))  # orthogonality check
print(len(log))
print(Q)

False
8.350821061200413e-16
35
[[-0.354  0.354  0.354 -0.354  0.354 -0.354 -0.354  0.354]
 [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354  0.354]
 [ 0.354 -0.354  0.354 -0.354  0.354 -0.354  0.354 -0.354]
 [ 0.354  0.354 -0.354 -0.354  0.354  0.354 -0.354 -0.354]
 [ 0.354 -0.354 -0.354  0.354  0.354 -0.354 -0.354  0.354]
 [ 0.354  0.354  0.354  0.354 -0.354 -0.354 -0.354 -0.354]
 [ 0.354 -0.354  0.354 -0.354 -0.354  0.354 -0.354  0.354]
 [ 0.354  0.354 -0.354 -0.354 -0.354 -0.354  0.354  0.354]]


In [36]:
# Example: complex 8x8 unitary
from scipy.stats import unitary_group
U = np.real(np.array([[0.354-0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,
   0.354+0.j,  0.354+0.j,  0.354+0.j],
 [0.354-0.j, -0.354+0.j,  0.354+0.j, -0.354-0.j,  0.354+0.j,
  -0.354-0.j,  0.354+0.j, -0.354-0.j],
 [0.354-0.j,  0.354-0.j, -0.354-0.j, -0.354+0.j,  0.354+0.j,
   0.354+0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j,  0.354+0.j,
  -0.354-0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j,  0.354-0.j,  0.354-0.j,  0.354-0.j, -0.354-0.j,
  -0.354-0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j, -0.354+0.j,
   0.354+0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j, 0.354-0.j, -0.354+0.j, -0.354+0.j, -0.354+0.j,
  -0.354+0.j,  0.354+0.j, 0.354+0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j,
   0.354+0.j,  0.354+0.j, -0.354+0.j]],  dtype=complex))
R, complex_rotations, rotations_mat = star_qr_complex_givens(U)

print("Upper triangular R (should be unitary if U is):")
print(np.round(R, 3))

print("\nComplex Givens rotations (central, target, c, s, col):")
for r in complex_rotations:
    print(r)


Upper triangular R (should be unitary if U is):
[[ 0.+0.j -0.+0.j -0.+0.j -0.+0.j  0.+0.j  0.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j -0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -0.+0.j -0.+0.j  0.+0.j -0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j  0.+0.j -0.+0.j -0.+0.j  0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j]]

Complex Givens rotations (central, target, c, s, col):
(1, 7, 0.7071067811865475, -0.7071067811865475, 0)
(1, 6, (-5.306679789904181e-17+0j), (-1+0j), 0)
(1, 5, (-0.7071067811865475+0j), (-0.7071067811865475+0j), 0)
(1, 4, (-5.306679789904181e-17+0j), (-1+0j), 0)
(1, 3, (-0.7071067811865475+0j), (-0.7071067811865475+0j), 0)
(1, 2, (-5.306679789904181e-17+0j), (-1+0j), 0)
(1, 0, (-0.7071067811865475+0j), (-0.707

In [42]:
# for r in rotations_mat:
#     coupling, f, phi = inverse_single_pulse(r)
couplings = []
phases = []
fractions = []
# print(len(rotation_mats))
for r in rotations_mat:
    coupling, f, phi = inverse_single_pulse(r)
    couplings.append(coupling)
    fractions.append(f)
    phases.append(phi/np.pi)
print('couplings = ',couplings)
print('fractions = ', fractions)
print('phases = ', phases)

couplings =  [(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (0, 1), (1, 5), (1, 4), (1, 3), (1, 2), (1, 7), (1, 6), (1, 5), (0, 1)]
fractions =  [0.5000000000000001, 1.0000000000000002, 1.5, 1.0000000000000002, 1.5, 1.0000000000000002, 1.5, 1.0, 1.5, 1.0, 1.5, 1.0, 1.5, 1.0, 1.5]
phases =  [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 0.5, 1.5, 0.5, 0.5, 0.5, 1.5, 0.5]


In [69]:
import numpy as np

def detect_adjacent_same_couplings(rotation_mats, tol=1e-8):

    matches = []
    for U1, U2 in zip(rotation_mats[:-1], rotation_mats[1:]):
        # Find which level (other than 0) this pulse couples with
        def find_coupled_level(U):
            for j in range(1, U.shape[0]):
                if np.abs(U[0, j]) > tol or np.abs(U[j, 0]) > tol:
                    return j
            return None

        j1 = find_coupled_level(U1)
        j2 = find_coupled_level(U2)

        matches.append(int(j1 == j2 and j1 is not None))

    return matches

import numpy as np
from numpy.linalg import norm

def compress_rotations(rotation_mats, central_node=0, tol=1e-8):
    i = 0
    optimized = []
    while i < len(rotation_mats):
        if i + 1 < len(rotation_mats):
            U1 = rotation_mats[i]
            U2 = rotation_mats[i+1]

            def find_coupled(U):
                for j in range(U.shape[0]):
                    if j == central_node:
                        continue
                    if np.abs(U[central_node, j]) > tol or np.abs(U[j, central_node]) > tol:
                        return j
                return None

            j1 = find_coupled(U1)
            j2 = find_coupled(U2)

            if j1 == j2 and j1 is not None:
                combined = U2 @ U1
                if norm(combined - np.eye(U1.shape[0])) < tol:
                    i += 2
                    continue
                else:
                    optimized.append(combined)
                    i += 2
                    continue

        optimized.append(rotation_mats[i])
        i += 1

    return optimized


import numpy as np
from numpy.linalg import norm
from math import acos, pi

def inverse_single_pulse(U, *, tol= 1e-8):

    U = np.asarray(U)
    dim = U.shape[0]

    # --- locate the only non‑trivial 2×2 block -----------------------------
    cand = [(p, q) for p in range(dim) for q in range(p+1, dim)
            if abs(U[p, q]) > tol or abs(U[q, p]) > tol]

    if len(cand) != 1:
        # raise ValueError("The matrix is not generated by a single two‑level pulse.")
        return (0,0), 0, 0

    i, j = cand[0]

    # everything outside rows/cols i,j must look like the identity
    mask = np.ones_like(U, dtype=bool)
    mask[[i, j], :] = False
    mask[:, [i, j]] = False
    if norm(U[mask] - np.eye(dim)[mask]) > tol:
        raise ValueError("Extra couplings detected – not a single pulse.")

    # --- extract rotation angle θ and phase φ -----------------------------
    c = U[i, i].real          # should equal U[j, j] and be real
    if abs(U[j, j].real - c) > tol or abs(U[i, i].imag) > tol or abs(U[j, j].imag) > tol:
        raise ValueError("Diagonal elements are inconsistent with a single pulse.")

    # numerical safety
    c = max(min(c, 1.0), -1.0)
    theta = 2 * acos(c)       # θ  (0 ≤ θ ≤ 2π)
    fraction = theta / pi

    # off‑diagonal element gives φ   (U_ij = –i sin(θ/2) e^{+iφ})
    u_ij = U[i, j]
    if abs(u_ij) < tol:
        raise ValueError("Zero off‑diagonal element – ambiguous phase.")
    phi = (np.angle(u_ij) + pi/2) % (2*pi)   # shift removes the –i factor

    return (i, j), fraction, phi

def givens_rotation(m, central, j, theta):
    G = np.eye(m)
    c = np.cos(theta)
    s = np.sin(theta)
    G[central, central] = c
    G[central, j] = s
    G[j, central] = -s
    G[j, j] = c
    return G


def star_qr_decomposition(order, U, central_node=0, tol=1e-10):
    m = U.shape[0]
    rotations_info = []  
    rotation_mats = []     
    V = U.copy()          

    col = 0
    count = 0
    for i in order[::-1]:
        a = V[central_node, col]
        b = V[i, col]
        if np.abs(b) < tol:
            continue
        theta = np.arctan2(b, a)
        G = givens_rotation(m, central_node, i, theta)
        V = G @ V
        rotations_info.append(("rotate", i, theta, col))
        rotation_mats.append(G)

    for col in order:
        theta_swap = np.pi / 2
        G_swap = givens_rotation(m, central_node, col, theta_swap)
        V = G_swap @ V
        rotations_info.append(("swap", col, theta_swap, col))
        rotation_mats.append(G_swap)
        count += 1 
        for i in order[count:]:
            a = V[central_node, col]
            b = V[i, col]
            if np.abs(b) < tol:
                continue
            theta = np.arctan2(b, a)
            G = givens_rotation(m, central_node, i, theta)
            V = G @ V
            rotations_info.append(("rotate", i, theta, col))
            rotation_mats.append(G)

        V = G_swap @ V   
        rotations_info.append(("swap_back", col, theta_swap, col))
        rotation_mats.append(G_swap)
    
    return rotations_info, V, rotation_mats




# dim = 5
# init_state = np.array([0, 0, 1, 0, 0])

# couplings = [(0, 2), (0, 3), (0, 1), (0, 4), (0, 1), (0, 3), (0, 2)]
# fractions = [1,
#              1.5,
#              2.0 * np.arcsin(np.sqrt(1/3)) / np.pi,
#              2/3,
#              2.0 * np.arcsin(np.sqrt(1/3)) / np.pi,
#              0.5,
#              1]
# fixed_phase_flags = [0, 1, 0, 1, 0, 1, 1]
# rabi_freqs = [1, 1, 1, 1, 1, 1, 1]

dim = 8
# init_state = np.array([0,1,0,0,0,0,0,0,0])
# couplings = [(2,0),(0,1),(2,0)] + [(4,0),(3,0)]  + [(0,1),(3,0)] + [(0,2),(4,0)]\
#           + [(6,0),(0,5),(6,0)] + [(8,0),(7,0)]  + [(0,5),(7,0)] + [(6,0)]\
#           + [(0,2),(6,0)] + [(5,0),(0,1),(5,0)] + [(7,0),(0,3),(7,0)] + [(0,4),(8,0)]
# fractions = [1,1/2,1] + [1,1/2]  + [1/2,1] + [1/2,1]\
#           + [1,1/2,1] + [1,1/2]  + [1/2,1] + [1/2]\
#           + [1/2,1] + [1,1/2,1] + [1,1/2,1] + [1/2,1]
# rabi_freqs = [1,1,1]+[1,1]+[1,1]+[1,1]\
#            + [1,1,1]+[1,1]+[1,1]+[1,1]\
#            +[1,1]+[1,1,1]+[1,1,1]+[1,1,1]
# fixed_phase_flags = [0.5,0.5,0.5] + [0.5,0.5]  + [0.5,0.5] + [0.5,0.5]\
#                    +[0.5,0.5,0.5] + [0.5,0.5]  + [0.5,0.5] + [0.5]\
#                    + [0.5,0.5] + [0.5,0.5,0.5] + [0.5,0.5,0.5] + [0.5,0.5]
    
# couplings, fixed_phase_flags = fix_couplings_and_phases(couplings, fixed_phase_flags)
# print("Fixed Couplings and Phase Flags:")
# print(couplings, fixed_phase_flags)

# U = np.real(unitary(couplings, rabi_freqs, fractions, fixed_phase_flags, dim))

U = np.real(np.array([[0.354-0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,
   0.354+0.j,  0.354+0.j,  0.354+0.j],
 [0.354-0.j, -0.354+0.j,  0.354+0.j, -0.354-0.j,  0.354+0.j,
  -0.354-0.j,  0.354+0.j, -0.354-0.j],
 [0.354-0.j,  0.354-0.j, -0.354-0.j, -0.354+0.j,  0.354+0.j,
   0.354+0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j,  0.354+0.j,
  -0.354-0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j,  0.354-0.j,  0.354-0.j,  0.354-0.j, -0.354-0.j,
  -0.354-0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j, -0.354+0.j,
   0.354+0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j, 0.354-0.j, -0.354+0.j, -0.354+0.j, -0.354+0.j,
  -0.354+0.j,  0.354+0.j, 0.354+0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j,
   0.354+0.j,  0.354+0.j, -0.354+0.j]],  dtype=complex))

# U = np.real(np.array([
#     [0.5-0.j,  0.5+0.j,  0.5-0.j,  0.5-0.j],
#     [0.5+0.j, -0.5-0.j,  0.5-0.j, -0.5+0.j],
#     [0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j],
#     [0.5+0.j, -0.5-0.j, -0.5-0.j,  0.5+0.j]
# ], dtype=complex))

import itertools

my_list = [1,4,2,3,5,6,7]
permutations = list(itertools.permutations(my_list))
central_node = 0
list_comp = []
len_comp = 36
for order in permutations:
    # print(order)
    rotations_info, V_triangular, rotation_mats = star_qr_decomposition(order,U, central_node)
    compressed = compress_rotations(rotation_mats)
    # coup, f, ph = inverse_single_pulse(compressed[13])
    # print(f)
    # if len(compressed) < len_comp:
    #     print(len(compressed))
        # list_comp.append(compressed)
        # len_comp = len(compressed)
    couplings = []
    phases = []
    fractions = []
    # print(len(rotation_mats))
    for r in compressed:
        coupling, f, phi = inverse_single_pulse(r)
        couplings.append(coupling)
        fractions.append(f)
        phases.append(phi/np.pi)
    # if couplings[0:6]==couplings[7:13][::-1]:
    # # if 3%np.round(fractions[0],2) == 0 and 3%np.round(fractions[12],2) == 0 and np.round(fractions[12],2) != 1:   
    # print('couplings = ',couplings)
    # print('fractions = ', fractions)
    # print('phases = ', phases)
# Display the sequence of rotations.
    # print("Sequence of rotations (each tuple: (operation, index, theta, column)):")
    # for op in rotations_info:
    #     print(op)
    
    # print("\nFinal matrix after applying rotations (ideally diagonal):")
    np.set_printoptions(precision=3, suppress=True)
    # print(V_triangular)
    
    U_reconstructed = np.eye(dim)
    for G in reversed(compressed[:]):
        U_reconstructed = G.T @ U_reconstructed
    
    # print("\nReconstructed U from the rotation matrices (should match the original U):")
    a = np.round(np.abs(U_reconstructed),3).tolist()
    b = np.round(np.abs(U),3).tolist()
    print(np.round(U_reconstructed,3))
    if len(couplings) < 30:
        print('____________________________This works_____________________________________')
        print(len(couplings))
        print('couplings = ',couplings)
        print('fractions = ', fractions)
        print('phases = ', phases)
print('done')

[[-0.354 -0.354 -0.354 -0.354 -0.354 -0.354  0.354  0.354]
 [-0.354  0.354 -0.354  0.354 -0.354  0.354  0.354 -0.354]
 [-0.354 -0.354  0.354  0.354 -0.354 -0.354 -0.354 -0.354]
 [-0.354  0.354  0.354 -0.354 -0.354  0.354 -0.354  0.354]
 [-0.354 -0.354 -0.354 -0.354  0.354  0.354 -0.354 -0.354]
 [-0.354  0.354 -0.354  0.354  0.354 -0.354 -0.354  0.354]
 [-0.354 -0.354  0.354  0.354  0.354  0.354  0.354  0.354]
 [-0.354  0.354  0.354 -0.354  0.354 -0.354  0.354 -0.354]]
[[-0.354 -0.354 -0.354 -0.354 -0.354 -0.354  0.354  0.354]
 [-0.354  0.354 -0.354  0.354 -0.354  0.354  0.354 -0.354]
 [-0.354 -0.354  0.354  0.354 -0.354 -0.354 -0.354 -0.354]
 [-0.354  0.354  0.354 -0.354 -0.354  0.354 -0.354  0.354]
 [-0.354 -0.354 -0.354 -0.354  0.354  0.354 -0.354 -0.354]
 [-0.354  0.354 -0.354  0.354  0.354 -0.354 -0.354  0.354]
 [-0.354 -0.354  0.354  0.354  0.354  0.354  0.354  0.354]
 [-0.354  0.354  0.354 -0.354  0.354 -0.354  0.354 -0.354]]
[[-0.354 -0.354 -0.354 -0.354 -0.354  0.354 -0.354  0.

KeyboardInterrupt: 